In [1]:
import tdmclient.notebook
import numpy as np
import sys
#sys.path.append(r'C:\Users\Usuario\OneDrive - epfl.ch\Documents\EPFL\Basics of mobile robotics\PROJET\Github\Mobile_robotics')
import math
import cv2 
import time
import importlib
import filters 
import computer_vision
import path_planning
import obstacle_avoidance

In [2]:
@tdmclient.notebook.sync_var
def set_speed(left, right):
    global motor_left_target, motor_right_target
    motor_left_target = left
    motor_right_target = right
    
@tdmclient.notebook.sync_var
def get_prox():
    global prox_horizontal
    obst = prox_horizontal
    return obst

In [95]:
# -*- coding: utf-8 -*-
"""
Created on Sun Nov 28 17:03:23 2021

@author: henry papapatos
"""


TRESH_DIST = 20 #mm
CAMERA = 0 # Camera
SPEED_TO_MMS = 0.3436 #150 speed
YAW_TO_MMS = 0.06086 # -100 100
SPEED_OFFSET = 5

def main():
         
    try:              
        frame_init = computer_vision.get_image(cap)
        # we need to wait a bit otherwise the image is yellow 
        time.sleep(1)
        
        print("take the rigth image")
        frame_init = computer_vision.get_image(cap)
        
        #frame_init = cv2.cvtColor(frame_init, cv2.COLOR_BGR2RGB)
        frame_init_crop = cv2.resize(frame_init,None, fx=0.5, fy= 0.5, interpolation = cv2.INTER_CUBIC)

        cv2.imshow('frame', frame_init_crop)
        
        cv2.waitKey(0)
        
        cv2.imwrite(r'C:\Users\papad\OneDrive\Images\Pellicule\img.jpg', frame_init) #POUR TUNING PAR ELIOTT
        
        # Extract vertexes, goals, thymio's start position and orientation from first frame
        start_pos, obst_vertexes, goals_pos, px_to_mm = computer_vision.Init(frame_init)
        
        print("Px_to_mm= ", px_to_mm)
        
        px_to_mm = 0.64
        
        computer_vision.display_obstacle(frame_init, start_pos, goals_pos, obst_vertexes)
        
        frame_init_crop = cv2.resize(frame_init,None, fx=0.5, fy= 0.5, interpolation = cv2.INTER_CUBIC)
        
        cv2.imshow('frame', frame_init_crop)
        
        cv2.waitKey(0)
        
        cv2.imwrite(r'C:\Users\papad\OneDrive\Images\Pellicule\img_obstacle.jpg', frame_init) #POUR TUNING PAR ELIOTT
        
        goal_list = path_planning.get_optimal_path(start_pos, goals_pos, obst_vertexes, 
                                                   px_to_mm, draw = True, image = frame_init)
        
        frame_init_crop = cv2.resize(frame_init,None, fx=0.5, fy= 0.5, interpolation = cv2.INTER_CUBIC)
        
        cv2.imwrite(r'C:\Users\papad\OneDrive\Images\Pellicule\img_path.jpg', frame_init) #POUR TUNING PAR ELIOTT

        cv2.imshow('frame', frame_init_crop)
        
        cv2.waitKey(0)
        
        frame = computer_vision.get_image(cap)
        
        # The estimated state vector at time t-1 in the global reference frame.
        # [x_t_minus_1, y_t_minus_1, yaw_t_minus_1]
        x_est_t_minus_1, hidden_cam, mask_frame = computer_vision.vision(frame, px_to_mm)
        # x_est_t_minus_1 = np.array([0.0,0.0,0.0])
        
        # The control input vector at time t-1 in the global reference frame.
        # [v, yaw_rate]
        u_t_minus_1 = np.array([0 ,0])
         
        # State covariance matrix P_t_minus_1
        P_t_minus_1 = np.array([[0.1,  0,   0],
                                [  0,0.1,   0],
                                [  0,  0, 0.1]])
    
    
        #update le dt car sinon kalman marche pas
        dt = 0.1 # EN SECONDES POUR KALMAN!!!!! same as PI_CLOCK of pid defined in filters?
        sum_error = 0 
        alt_error_pid = 0
        v_l = 0
        v_r = 0
        
        state = 0
        obstThrH = 2000
        
        time.sleep(0.1)
        
        while len(goal_list)!=0:
            
            #print("goal is: ", goal_list[0])
            
            previous = time.time()
            
            frame = computer_vision.get_image(cap)
            
            obs_vector_z_t, hidden_cam, mask_frame = computer_vision.vision(frame, px_to_mm)
            
            
            computer_vision.display_obstacle(frame, start_pos, goals_pos, obst_vertexes)
            
            computer_vision.display_pos(frame,obs_vector_z_t, px_to_mm, hidden_cam, is_from_camera = True)
            
            
#             print(f'Timestep measurement={obs_vector_z_t}')
    
            # Run the Extended Kalman Filter and store the 
            # near-optimal state and covariance estimates
            optimal_state_estimate_t, covariance_estimate_t = filters.ekf(
                obs_vector_z_t, # Most recent sensor measurement
                x_est_t_minus_1, # Our most recent estimate of the state
                u_t_minus_1, # Our most recent control input
                P_t_minus_1, # Our most recent state covariance matrix
                dt,hidden_cam) # Time interval
            
            computer_vision.display_pos(frame, optimal_state_estimate_t, px_to_mm, 0, is_from_camera = False)
            computer_vision.display_pos(frame, goal_list[0], px_to_mm, 0, is_from_camera = 2)
            
            frame = cv2.resize(frame,None, fx=0.5, fy= 0.5, interpolation = cv2.INTER_CUBIC)
            cv2.imshow('running frame', frame)
            if cv2.waitKey(1) == ord('q'):
                break
            
            mask_frame = cv2.resize(mask_frame,None, fx=0.5, fy= 0.5, interpolation = cv2.INTER_CUBIC)
            cv2.imshow('mask frame', mask_frame)
            if cv2.waitKey(1) == ord('q'):
                break
                
            if np.linalg.norm(np.array([optimal_state_estimate_t[0],optimal_state_estimate_t[1]])-np.array(goal_list[0])) < TRESH_DIST:
                goal_list.pop(0)
                
            # Get ready for the next timestep by updating the variable values
            x_est_t_minus_1 = optimal_state_estimate_t
            P_t_minus_1 = covariance_estimate_t
            
            #check if obstacle in coming
            obst=get_prox()
            
            if state == 0:
                for i in range(len(obst)-2):
            # switch from goal tracking to obst avoidance if obstacle detected
                    if (obst[i] > obstThrH):
                        print('obst', obst)
                        state = 1
            if state==0:
                sum_error, alt_error_pid, v_l,v_r = filters.pid(optimal_state_estimate_t,goal_list[0],sum_error, alt_error_pid,dt)
    
            else:  #if yes, obstacle avoidance
                v, state = obstacle_avoidance.obstacle_avoidance(obst, v_l, v_r, True)
                v_l = v[0]
                v_r = v[1]
            
            v = (v_l + v_r)*SPEED_TO_MMS/2*1.1
            yaw = (v_l-v_r)*YAW_TO_MMS*4
            u_t_minus_1 = [v, yaw]; # delta_v returned by PID
                
            set_speed(int(v_l), int(v_r+SPEED_OFFSET))

            actual = time.time()
            diff = actual-previous
#             print("elapsed time is: ", diff)
            if diff<dt:
                time.sleep(dt-diff)
            
    finally: 
        set_speed(0, 0)
        cv2.destroyAllWindows()



In [4]:
print("connecting to thymio")
await tdmclient.notebook.start()

connecting to thymio


In [5]:
print('Connecting to camera...')
cap=cv2.VideoCapture(CAMERA) # Either 0 or 1, front camera or external cam
print('Camera connected !')

Connecting to camera...
Camera connected !


In [6]:
#get the full quality of the camera
print("setting parameters...")
cap.set(3,1920) 
print("first parameter set")
cap.set(4,1080)
print("second parameter set")

setting parameters...
first parameter set
second parameter set


In [92]:
importlib.reload(filters)
importlib.reload(path_planning)
importlib.reload(computer_vision)
importlib.reload(obstacle_avoidance)

<module 'obstacle_avoidance' from 'C:\\Users\\papad\\OneDrive\\Documents\\MA1\\Basic_of_mobile_robotics\\project\\Mobile_robotics\\obstacle_avoidance.py'>

In [97]:
main()

take the rigth image
Px_to_mm=  0.46528856319736445


100%|██████████| 1/1 [00:00<00:00, 73.28it/s]


angle du robot 223.05851360908562
angle du robot 222.63273649383225
angle du robot 223.15476078505904
angle du robot 227.20643135660583
angle du robot 239.0263867161037
angle du robot 245.68010959858842
angle du robot 256.88591163173083
angle du robot 272.83412853616505
angle du robot 277.9292448388359
angle du robot 285.3163857823621
angle du robot 291.9903268879659
angle du robot 298.3046595528616
angle du robot 300.6205472237866
angle du robot 306.7238192719701
angle du robot 309.57081842110773
angle du robot 312.27091164441316
angle du robot 315.48143595216345
angle du robot 317.4303744082327
angle du robot 321.44246370681
angle du robot 323.9245338184523
angle du robot 325.45448387931975
angle du robot 328.59986811464086
angle du robot 330.72240489719616
angle du robot 332.5287982678704
angle du robot 333.751947476159
angle du robot 335.4643999884825
angle du robot 336.4610209520183
angle du robot 338.1401652427886
angle du robot 338.83563060217205
angle du robot 341.4044098686045

angle du robot 69.08736272685277
angle du robot 83.61315831979299
angle du robot 74.25892163828814
angle du robot 65.81556310788152
angle du robot 58.194385057731765
angle du robot 304.4142767157193
angle du robot 292.6229958896109
angle du robot 121.23714861382399
angle du robot 36.36262990773869
angle du robot 27.014983177268178
angle du robot 21.460114159692612
angle du robot 18.226692945709214
angle du robot 14.767356944735393
angle du robot 13.124483546033254
angle du robot 9.15928889682284
angle du robot 6.899369551318438
angle du robot 4.850886334473746
angle du robot 2.8069719351441966
angle du robot 1.373945773316806
angle du robot 329.17982111541045
angle du robot 355.7035093278106
angle du robot 357.7556609532834
angle du robot 356.6210765905974
angle du robot 355.29124861697875
angle du robot 355.18976961377814
angle du robot 353.9433239267316
angle du robot 353.847617829561
angle du robot 353.4360969016894
angle du robot 353.191303582139
angle du robot 352.55711209964295
a

obst [0, 0, 3392, 0, 0, 0, 0]
remanent left speed:  -6.0
remanent right speed:  -6.0
left speed:  -6.0
right speed:  -6.0
angle du robot 349.34495678850607
obst [0, 0, 3392, 0, 0, 0, 0]
remanent left speed:  -6.0
remanent right speed:  -6.0
left speed:  -6.0
right speed:  -6.0
angle du robot 349.9377184968388
obst [0, 0, 3391, 0, 0, 0, 0]
remanent left speed:  -6.0
remanent right speed:  -6.0
left speed:  -6.0
right speed:  -6.0
angle du robot 349.9874631879723
obst [0, 0, 3391, 0, 0, 0, 0]
remanent left speed:  -6.0
remanent right speed:  -6.0
left speed:  -6.0
right speed:  -6.0
angle du robot 349.99163777324185
obst [0, 0, 3391, 0, 0, 0, 0]
remanent left speed:  -6.0
remanent right speed:  -6.0
left speed:  -6.0
right speed:  -6.0
angle du robot 349.6138921189138
obst [0, 0, 3381, 0, 0, 0, 0]
remanent left speed:  -6.0
remanent right speed:  -6.0
left speed:  -6.0
right speed:  -6.0
angle du robot 349.9602876080988
obst [0, 0, 3381, 0, 0, 0, 0]
remanent left speed:  -6.0
remanent ri

angle du robot 315.0939512787444
angle du robot 320.0832702794435
angle du robot 324.7992367986773
angle du robot 327.6602152624767
angle du robot 331.6102495542216
angle du robot 334.628333313143
angle du robot 336.7730928082493
angle du robot 339.35477434588654
angle du robot 341.6850620621148
angle du robot 343.78843568168764
angle du robot 345.68699119168093
angle du robot 347.4006729603331
angle du robot 348.94748325308103
obst [0, 0, 0, 0, 3585, 0, 0]
remanent left speed:  14.338113022523885
remanent right speed:  -14.338113022523885
left speed:  14.338113022523885
right speed:  -14.338113022523885
angle du robot 350.34367134676233
obst [0, 0, 0, 0, 3585, 0, 0]
remanent left speed:  -12.5
remanent right speed:  62.5
left speed:  -12.5
right speed:  62.5
angle du robot 346.69207134676236
remanent left speed:  -26.0
remanent right speed:  101.0
left speed:  -26.0
right speed:  101.0
angle du robot 340.50869534676235
remanent left speed:  -40.5
remanent right speed:  128.0
left spee

angle du robot 0.9513396544742516
angle du robot 5.016025840196909
angle du robot 0.7288394591161023
angle du robot 4.782982605244039
angle du robot 8.442349508374623
angle du robot 11.745381899945961
angle du robot 14.72677820935565
angle du robot 17.417858071740262
angle du robot 19.846891341445307
angle du robot 22.039395067479553
angle du robot 24.01840155068749
angle du robot 25.80470029858657
angle du robot 27.41705641961023
angle du robot 28.87240775099309
angle du robot 30.186042791131214
angle du robot 31.37176130560085
angle du robot 32.44201929400548
angle du robot 33.40805984053123
angle du robot 34.28726599934998
angle du robot 35.08749805475104
angle du robot 35.81589982801555
angle du robot 36.47896455822557
angle du robot 102.91566977032281
angle du robot 108.60999401863263
angle du robot 105.64390841685179
angle du robot 100.31629047353854
angle du robot 93.99673284247928
angle du robot 86.48068398291106
angle du robot 83.09074495223462
angle du robot 77.09312811004548

remanent left speed:  134.0
remanent right speed:  -32.5
left speed:  134.0
right speed:  -32.5
angle du robot 57.57545367932478
remanent left speed:  118.5
remanent right speed:  -18.0
left speed:  118.5
right speed:  -18.0
angle du robot 272.2991007705664
remanent left speed:  111.0
remanent right speed:  -10.5
left speed:  111.0
right speed:  -10.5
angle du robot 88.90713392199012
remanent left speed:  80.5
remanent right speed:  19.5
left speed:  80.5
right speed:  19.5
angle du robot 77.83752244089833
angle du robot 273.9425379041644
angle du robot 100.8311125457555
angle du robot 92.02505579699722
angle du robot 90.79089182816043
angle du robot 87.31167223272959
angle du robot 82.6396941832086
angle du robot 78.2607492457497
angle du robot 74.39539730978726
angle du robot 67.21576854203315
angle du robot 64.42625097963628
angle du robot 60.045409935343564
angle du robot 55.76186101304466
angle du robot 92.11755313828122
angle du robot 54.235048865110926
angle du robot 49.49459203

angle du robot 244.012428730613
angle du robot 244.23786948341044
angle du robot 244.01157060207663
angle du robot 243.74443967008497
angle du robot 244.23341246111548
angle du robot 243.76001163260977
angle du robot 244.4783230906204
angle du robot 244.2892144738601
angle du robot 244.27315822364378
angle du robot 244.78536681999995
angle du robot 244.30885668744156
angle du robot 244.5401142168738
angle du robot 244.28944831760433
angle du robot 244.7801272410115
angle du robot 244.5709466650181
angle du robot 244.2833592689973
angle du robot 244.5267601491928
angle du robot 244.0199562739935
angle du robot 244.49349671376766
angle du robot 244.00975946608168
angle du robot 243.1902366472878
angle du robot 243.63247732697903
angle du robot 242.3620023045024
angle du robot 241.69440865073238
angle du robot 241.1338506470605
angle du robot 240.30633408107062
angle du robot 239.45198041396003
angle du robot 238.590584098334
angle du robot 238.0303253155361
angle du robot 236.87029753140

IndexError: list index out of range

In [ ]:
cap.release()

In [ ]:
await tdmclient.notebook.stop()